# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [ ]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv')

import functions, importlib
importlib.reload(functions)

df_normalized = functions.normalize_column_names(df)
df_cleaned = functions.clean_invalid_values(df_normalized)
main_df = functions.remove_null_values(df_cleaned)

main_df.head(5)

Column names normalized
Inconsistent values replaced
Cleaning 7 columns with null or missing values
All null values have been successfully handled


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.000000,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.000000,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.000000,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.000000,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,0.384256,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [7]:
condition = (main_df['response'] == 'Yes') & (main_df['total_claim_amount'] > 1000)
golden_customers = main_df[condition]
golden_customers.head(5)

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury,Medsize,A
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [18]:
# Filtrar solo los clientes que respondieron "Yes" a la campaña de marketing
condition2 = main_df['response'] == 'Yes'
responding_customers = main_df[condition2]

# Usar groupby para calcular el promedio del total_claim_amount por policy_type y gender
average_claims = responding_customers.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().round(2)

# Convertir a DataFrame para una mejor visualización
average_claims_df = average_claims.reset_index()

average_claims_df

,policy_type,gender,total_claim_amount
0,Corporate Auto,F,433.74
1,Corporate Auto,M,408.58
2,Personal Auto,F,452.97
3,Personal Auto,M,457.01
4,Special Auto,F,453.28
5,Special Auto,M,429.53


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [21]:
# Agrupar por estado y contar clientes
customers_by_state = main_df.groupby('state')['customer'].nunique()

# Filtrar para mostrar solo estados con más de 500 clientes
states_with_many_customers = customers_by_state[customers_by_state > 500]

# Convertir a DataFrame (opcional, para mejor manejo y visualización)
states_with_many_customers_df = states_with_many_customers.reset_index()

# Mostrar resultados
print(states_with_many_customers_df)

        state  customer
0     Arizona      1703
1  California      3520
2      Nevada       882
3      Oregon      2601
4  Washington       798


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [23]:
group_stats = responding_customers.groupby(['education', 'gender'])['customer_lifetime_value'].describe().round(2)
group_stats


count      mean      std      min      25%  \
education            gender                                               
Bachelor             F       230.0   7377.74  6329.71  2248.45  4118.59   
                     M       193.0   8647.85  6846.12  2300.69  3567.50   
College              F       201.0   7083.12  5688.31  2004.35  3746.75   
                     M       256.0   7440.37  4881.61  2393.92  4270.03   
Doctor               F        28.0   4501.96  1482.20  2395.57  3609.73   
                     M        37.0   6544.02  4115.47  2491.32  3436.13   
High School or Below F       207.0  10679.83  9876.56  2397.04  4250.28   
                     M       183.0   6850.48  2670.37  2227.07  5325.72   
Master               F        78.0   7932.82  5410.05  2574.02  4170.69   
                     M        53.0   7738.72  3970.69  2551.23  5479.56   

                                 50%       75%       max  
education            gender                               
Bachelor             F       5246.28   8427.17  33473.35  
                     M       5548.03  12036.62  24127.50  
College              F       4834.71   8002.31  25807.06  
                     M       5989.77   8717.05  22332.44  
Doctor               F       4673.46   5565.68   6265.34  
                     M       5073.28  10972.07  12731.95  
High School or Below F       6503.40  13770.98  41787.90  
                     M       6366.23   8819.02  12298.69  
Master               F       5096.67  11981.98  19160.99  
                     M       8509.85   9075.77  14435.67

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here